In [1]:
import scipy
import numpy as np
from itertools import product
from matplotlib import pyplot as plt
from scipy.optimize import LinearConstraint, Bounds

In [2]:
shipping_cost = np.array([[1,3],[3,2],[2,2]]).reshape(3,2)
production_cost = np.array([6,5]).reshape(1,2)
given_demand = np.array([[50,60,75],[75,90,100],[60,75,90]]).reshape(3,3)
produce_limit = np.array([250,250]).reshape(2,1)

In [3]:
Gamma = 1
# Gamma = 2

In [4]:
products = (list(product(range(3),repeat=3)))
senarios = [[given_demand[i][j]  for i,j in enumerate(pro)] for pro in products]

destruct_senarios = np.random.binomial(1,0.3,(1,3,2))
destruct_units = np.where(destruct_senarios == 1, 0.5, 1)

num_senario = len(senarios)
num_dest_senario = len(destruct_senarios)

#create empty numpy array
Resource_lft = np.array([])
demand_array = np.array([])

for demand in senarios:
    for destruct in destruct_units:
        transport_mat = np.zeros((3,6))

        for ind,i in enumerate(demand):
            transport_mat[ind,2*ind] = destruct[ind][0]
            transport_mat[ind,2*ind+1] = destruct[ind][1]
            demand_array =  np.append(demand_array,i)

        Resource_lft = np.append(Resource_lft,transport_mat)
Resource_lft = Resource_lft.reshape(num_dest_senario*num_senario*3,6)


# Shortfall Matrix 
Shortfall_lft = np.eye(num_senario*num_dest_senario*3)

# Allocation Matrix
Resource_lft = Resource_lft.reshape(num_dest_senario*num_senario*3,6)

# Demand Matrix
demand_rgt = demand_array.reshape(num_dest_senario*num_senario*3,1)

# Production Matrix
Production_lft = np.repeat([[0,0],[0,0],[0,0]],num_dest_senario*num_senario,axis=0)

# Constraint Matrix
A1_ub = -1*np.concatenate((Production_lft,Resource_lft,Shortfall_lft),axis=1)
b1_ub = -1*demand_rgt

# Balance Matrix
Balance_lft = np.array([[-1,0,1,0,1,0,1,0],[0,-1,0,1,0,1,0,1]])
Dummy_lft = np.zeros((2,len(Shortfall_lft)))

# Concatenate
A2_ub = np.concatenate((Balance_lft,Dummy_lft),axis=1)
b2_ub = np.array([0,0]).reshape(2,1)

A_combined = np.concatenate((A1_ub,A2_ub),axis=0)
b_combined = np.concatenate((b1_ub,b2_ub),axis=0)

A_ex= np.zeros((len(A_combined),num_senario*num_dest_senario))

A_combined = np.concatenate((A_combined,A_ex),axis=1)


In [5]:
print(list(A_combined))

[array([-0. , -0. , -1. , -0.5, -0. , -0. , -0. , -0. , -1. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. , -0. ,
       -0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
        0. ,  0. ,  0. ,  0. ,  0. ,  0. ]), array([-0., -0., -0., -0., -1., -1., -0., -0., -0., -1., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -

In [6]:
A1_ub.shape

(81, 89)

In [7]:
A2_ub.shape

(2, 89)

In [8]:
A_combined.shape

(83, 116)

In [9]:
A1_ub.shape

(81, 89)

In [10]:
A_combined[-2:]

array([[-1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [11]:
A_combined

array([[-0., -0., -1., ...,  0.,  0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       ...,
       [-0., -0., -0., ...,  0.,  0.,  0.],
       [-1.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0., -1.,  0., ...,  0.,  0.,  0.]])

In [12]:
new_row =[]
for i in production_cost:
    new_row.extend(i)

for i in shipping_cost:
    for j in i:
        new_row.extend([j])

deploy_move_cost = np.repeat([new_row],num_senario*num_dest_senario,axis=0)

new_unstatisfied_demand_costs = np.kron(40*np.eye(num_dest_senario*num_senario, dtype=int), np.ones((1, 3), dtype=int))

overall_cost = np.eye(num_senario*num_dest_senario)

A3_ub = np.concatenate((-1*deploy_move_cost,-1*new_unstatisfied_demand_costs,overall_cost),axis=1)
b3_ub = np.zeros(len(A3_ub)).reshape(-1,1)

In [13]:
print(A3_ub.shape)

(27, 116)


In [14]:
print(A3_ub)

[[-6. -5. -1. ...  0.  0.  0.]
 [-6. -5. -1. ...  0.  0.  0.]
 [-6. -5. -1. ...  0.  0.  0.]
 ...
 [-6. -5. -1. ...  1.  0.  0.]
 [-6. -5. -1. ...  0.  1.  0.]
 [-6. -5. -1. ...  0.  0.  1.]]


In [15]:
A_ub = np.concatenate((A_combined,A3_ub),axis=0)
b_ub = np.concatenate((b_combined,b3_ub),axis=0)

In [16]:
A3_ub[0]

array([ -6.,  -5.,  -1.,  -3.,  -3.,  -2.,  -2.,  -2., -40., -40., -40.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.])

In [17]:
new_unstatisfied_demand_costs.shape

(27, 81)

In [18]:
bounds = []

for i in produce_limit:
    bounds.append([0,i[0]])

for i in shipping_cost:
    for j in i:
        bounds.append([0,250])

for i in range(3*num_senario*num_dest_senario):
    bounds.append([0,float("inf")])

for i in range(num_senario*num_dest_senario):

    bounds.append([0,float("inf")])

In [19]:
def utility_val(x):             # calculates the value and the gradient of the expected utility function
    
    # utility function realizations, will be used for Hessian calculations

    length = len(x)-num_dest_senario*num_senario
    
    Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
    val = sum(Realizations)/num_dest_senario*num_senario
    gradient = (Gamma/num_dest_senario*num_senario) * Realizations
    gradient = np.append(np.zeros(length),gradient,axis=0)
    return (val, gradient)
         
def utility_Hp(x,p):            # returns Hessian of the expected utility function multiplied by a vector p
                                # this is easy because the Hessian is diagonal
    Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
    Hp =  Realizations * p[-num_dest_senario*num_senario:] * (Gamma**2/num_dest_senario*num_senario)
    length = len(x)-num_dest_senario*num_senario
    Hp = np.append(np.zeros(length),Hp,axis=0)
    return Hp

In [20]:
# def utility_val(x):             # calculates the value and the gradient of the expected utility function
    
#     # utility function realizations, will be used for Hessian calculations

#     length = len(x)-num_dest_senario*num_senario
    
#     Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
#     val = sum(Realizations)/num_dest_senario*num_senario
#     gradient = (Gamma/num_dest_senario*num_senario) * Realizations
#     gradient = np.append(np.zeros(length),gradient,axis=0)
#     return (val)
         
# def utility_Hp(x,p):            # returns Hessian of the expected utility function multiplied by a vector p
#                                 # this is easy because the Hessian is diagonal
#     Realizations = np.exp(Gamma*x[-num_dest_senario*num_senario:])
#     Hp =  Realizations * p[-num_dest_senario*num_senario:] * (Gamma**2/num_dest_senario*num_senario)
#     length = len(x)-num_dest_senario*num_senario
#     Hp = np.append(np.zeros(length),Hp,axis=0)
#     return Hp

In [21]:
l_ub = np.full((len(b_ub),),0.0)
l_ub[-num_dest_senario*num_senario-2:-num_dest_senario*num_senario] = -np.inf
l_ub[-num_dest_senario*num_senario:] = -1

In [22]:
len(l_ub[(-num_dest_senario*num_senario) -2:-num_dest_senario*num_senario])

2

In [23]:
constraints = LinearConstraint(A_ub, l_ub.reshape(-1), b_ub.reshape(-1))
bounds = Bounds(np.array(bounds)[:,0], np.array(bounds)[:,1])

In [24]:
z0 = np.zeros((len(A_ub[0])))

In [25]:
len(z0)

116

In [26]:
SolutionVec = scipy.optimize.minimize(utility_val, z0, method='trust-constr', jac=True, hessp=utility_Hp,
               constraints = constraints, options={'verbose': 1,'maxiter': 1000}, bounds=bounds)

c:\Users\aayus\Documents\GitHub\StochOpt\.venv\lib\site-packages\scipy\optimize\_trustregion_constr\equality_constrained_sqp.py:217: UserWarning: Singular Jacobian matrix. Using SVD decomposition to perform the factorizations.
  Z, LS, Y = projections(A, factorization_method)


KeyboardInterrupt: 

In [ ]:
# SolutionVec = scipy.optimize.minimize(utility_val, z0, method='LBFGS',
#                constraints = constraints, options={'verbose': 1,'maxiter': 500}, bounds=bounds)

In [ ]:
SolutionVec.x

In [ ]:
b_ub.shape